# REGRESIÓN LOGIT Y PROBIT
## Realizado por: Pablo Sánchez Cabrera

- Carga de las librerías y los datos

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf # creación del modelo con sintaxisis de R 
from statsmodels.genmod.families import links  # para obtener links de GLM
from statsmodels.discrete.discrete_model import DiscreteResults # estadísticos para análisis bondad de ajuste del modelo
from statsmodels.genmod.generalized_linear_model import GLMResultsWrapper # tipado de los modelos GLM (wrapper R)

In [2]:
datos = pd.read_csv("../data/datos.csv", sep=",", encoding = "ISO-8859-1")

Se plantea realizar dos modelos de clasificación (variable dicotómica) con dos tipos de regresión que forman parte del **Modelo Lineal Generalizado**:
- Regresión Logística
- Regresión Probit

Las variables explicativas a tener en cuenta son las siguientes:ç

- nmiemb: número de miembros del hogar
- nmiem11: número de miembros del hogar entre 35-64 años
- numinacti: número de miembros activos en el hogar
- numocu: número de miembros ocupados en el hogar
- tiphogar1: tipo de hogar - tipo de clasificación
- situocuhog: situación del hogar respecto a la ocupación

Por su parte, la variable `impexac`, ingresos corrientes del hogar, será discretizada en 2 para obtener una variable dicotómica (pobre vs rico). Así, esta variable se define como:
- 1 (pobre) si ingresos per cápita < 60% de ingresos per cápita
- 0 (rico) en otro caso

In [3]:
datos['ingpc'] = datos['impexac']/datos['nmiemb'] #ingresos per cápita - 
umbral = np.median(datos['ingpc'])*0.6
datos['respuesta'] = np.where(datos['ingpc']<umbral,1,0)

# Creación data.frame con las variables a utilizar
variables = ['nmiemb', 'nmiem11', 'numinacti', 'numocu', 'tiphogar1', 'situocuhog', 'respuesta']
datos = datos.loc[:, variables]

Se analiza de forma rápida el dataset final así como la distribución del target

In [4]:
datos.shape #tamaño df
datos.head(6)

,nmiemb,nmiem11,numinacti,numocu,tiphogar1,situocuhog,respuesta
0,4,2,1,2,Pareja con al menos un hijo de 16 o más años,El sustentador principal y el cónyuge ocupados...,0
1,2,0,0,1,Pareja sin hijos teniendo los dos miembros men...,"El sustentador principal o el cónyugeocupado, ...",0
2,1,0,1,0,Una persona de 65 o más años,Ningún ocupado en el hogar,1
3,3,2,0,2,Pareja con un hijo menor de 16 años,El sustentador principal y el cónyuge ocupados...,0
4,3,2,0,1,Pareja con un hijo menor de 16 años,"El sustentador principal o el cónyugeocupado, ...",0
5,1,1,1,0,Una persona de 30 a 64 años,Ningún ocupado en el hogar,0


In [5]:
# Variable respuesta - Frecuencia
pd.crosstab(datos['respuesta'], columns='frecuencia').apply(lambda p: p/p.sum(), axis=0) #tabla cruzada (en porcentaje)

col_0,frecuencia
respuesta,
0,0.775174
1,0.224826


A la vista de los resultados, se observa que la muestra no está balanceada (más registros con respuesta cero que uno)

Vamos a iniciar los modelos de `regresión logística` y `regresión probit`. 

Antes de realizar la codificación de ambos modelos se define la fórmula de trabajo (similar a *R*).

In [6]:
formula = 'respuesta ~ nmiemb + nmiem11 + numinacti + numocu + C(tiphogar1) + C(situocuhog)'  #C para identificar el factor

## Modelo de Regresión Logística

In [7]:
logit = smf.glm(formula, data=datos, family=sm.families.Binomial()).fit() 
logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:              respuesta   No. Observations:                18882
Model:                            GLM   Df Residuals:                    18859
Model Family:                Binomial   Df Model:                           22
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -7385.7
Date:                Sat, 02 Dec 2023   Deviance:                       14771.
Time:                        11:29:48   Pearson chi2:                 2.44e+04
No. Iterations:                     6   Pseudo R-squ. (CS):             0.2684
Covariance Type:            nonrobust                                         
===============================================================================================================================================================================================
                                                                                                                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                                      -2.7081      0.871     -3.109      0.002      -4.415      -1.001
C(tiphogar1)[T.Padre o madre solo, con al menos un hijo de 16 o más años]                                                       0.5893      0.132      4.478      0.000       0.331       0.847
C(tiphogar1)[T.Pareja con al menos un hijo de 16 o más años]                                                                    0.5113      0.083      6.163      0.000       0.349       0.674
C(tiphogar1)[T.Pareja con dos hijos menores de 16 años]                                                                        -0.3840      0.106     -3.615      0.000      -0.592      -0.176
C(tiphogar1)[T.Pareja con tres o más hijos menores de 16 años]                                                                 -0.6002      0.165     -3.638      0.000      -0.923      -0.277
C(tiphogar1)[T.Pareja con un hijo menor de 16 años]                                                                             0.2479      0.114      2.170      0.030       0.024       0.472
C(tiphogar1)[T.Pareja sin hijos teniendo al menos uno de los miembros 65 años o más]                                           -0.4082      0.121     -3.364      0.001      -0.646      -0.170
C(tiphogar1)[T.Pareja sin hijos teniendo los dos miembros menos de 65 años]                                                     0.2108      0.128      1.641      0.101      -0.041       0.462
C(tiphogar1)[T.Un adulto con niños menores de 16 años]                                                                          0.7952      0.229      3.479      0.001       0.347       1.243
C(tiphogar1)[T.Una persona de 30 a 64 años]                                                                                     0.6961      0.158      4.409      0.000       0.387       1.006
C(tiphogar1)[T.Una persona de 65 o más años]                                                                                   -1.0251      0.175     -5.871      0.000      -1.367      -0.683
C(tiphogar1)[T.Una persona de menos de 30 años]                                                                                 1.5050      0.355      4.242      0.000       0.810       2.200
C(situocuhog)[T.El sustentador principal o el cónyugeocupado, al menos otros dos miembros ocupados]                            -0.3353      0.600     -0.559      0.576      -1.512       0.841
C(situocuho

## Modelo de Regresión Probit

In [8]:
probit = smf.glm(formula, data=datos, family=sm.families.Binomial(links.Probit())).fit() 
probit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:              respuesta   No. Observations:                18882
Model:                            GLM   Df Residuals:                    18859
Model Family:                Binomial   Df Model:                           22
Link Function:                 Probit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -7397.9
Date:                Sat, 02 Dec 2023   Deviance:                       14796.
Time:                        11:29:48   Pearson chi2:                 6.85e+05
No. Iterations:                     7   Pseudo R-squ. (CS):             0.2675
Covariance Type:            nonrobust                                         
===============================================================================================================================================================================================
                                                                                                                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                                      -1.5204      0.450     -3.376      0.001      -2.403      -0.638
C(tiphogar1)[T.Padre o madre solo, con al menos un hijo de 16 o más años]                                                       0.2825      0.078      3.634      0.000       0.130       0.435
C(tiphogar1)[T.Pareja con al menos un hijo de 16 o más años]                                                                    0.2854      0.048      5.990      0.000       0.192       0.379
C(tiphogar1)[T.Pareja con dos hijos menores de 16 años]                                                                        -0.2589      0.061     -4.246      0.000      -0.378      -0.139
C(tiphogar1)[T.Pareja con tres o más hijos menores de 16 años]                                                                 -0.3587      0.094     -3.799      0.000      -0.544      -0.174
C(tiphogar1)[T.Pareja con un hijo menor de 16 años]                                                                             0.1073      0.065      1.642      0.101      -0.021       0.235
C(tiphogar1)[T.Pareja sin hijos teniendo al menos uno de los miembros 65 años o más]                                           -0.2838      0.068     -4.159      0.000      -0.418      -0.150
C(tiphogar1)[T.Pareja sin hijos teniendo los dos miembros menos de 65 años]                                                     0.0693      0.073      0.951      0.342      -0.074       0.212
C(tiphogar1)[T.Un adulto con niños menores de 16 años]                                                                          0.4250      0.133      3.187      0.001       0.164       0.686
C(tiphogar1)[T.Una persona de 30 a 64 años]                                                                                     0.2920      0.092      3.189      0.001       0.113       0.471
C(tiphogar1)[T.Una persona de 65 o más años]                                                                                   -0.5916      0.092     -6.437      0.000      -0.772      -0.411
C(tiphogar1)[T.Una persona de menos de 30 años]                                                                                 0.8071      0.218      3.708      0.000       0.381       1.234
C(situocuhog)[T.El sustentador principal o el cónyugeocupado, al menos otros dos miembros ocupados]                            -0.2086      0.312     -0.670      0.503      -0.819       0.402
C(situocuho

## Predicción de modelos y bondad de ajuste 

Una vez realizados los modelos se pueden obtener las predicciones del modelo así como analizar su bondad de ajuste.

### Matriz de confusión

In [9]:
def analisis_predicciones(modelo: GLMResultsWrapper, datos_reales: pd.DataFrame, threshold: float = 0.5):
    """
    Matriz de confusión de y porcentaje medio del número de aciertos del modelo

    Parameters
    ----------
    modelo : tipado del modelo (wrapper R)
    datos_reales: dataframe con el valor de la variable objetivo 
    threshold: umbral de probabilidad para tomar la decisión de instancia positiva (default 0.5)

    Returns
    -------
    matriz_confusion: dataframe con los resultados de las predicciones vs datos reales
    """
    pred = pd.DataFrame(modelo.predict(datos), columns=['prediccion'])  #predicción con los datos utilizados para crear el modelo
    df = pd.concat([datos['respuesta'], pred], axis=1) #data.frame respuesta y predicción
    print(f"Registros sin predicción: {np.array(pred.isna().sum())[0]}")
    df = df.dropna(axis=0) #elimina registros que tienen NaN ->(nº observaciones: 18882)
    
    # si la probabilidad es mayor que el threshold es igual a 1, 0 en otro caso
    df['prediccion'] = np.where(df['prediccion'] > threshold, 1, 0)
    
    matriz_confusion = pd.crosstab(df['respuesta'], df['prediccion'])
    print(f"Aciertos del modelo (% medio): {round(np.mean(df['prediccion'] == df['respuesta']), 5)}")
    return matriz_confusion

- Modelo Logit

In [10]:
analisis_predicciones(logit, datos)

Registros sin predicción: 3264
Aciertos del modelo (% medio): 0.82248


prediccion,0,1
respuesta,,
0,13480,930
1,2422,2050


- Modelo Probit

In [11]:
analisis_predicciones(probit, datos)

Registros sin predicción: 3264
Aciertos del modelo (% medio): 0.82226


prediccion,0,1
respuesta,,
0,13495,915
1,2441,2031


#### Deviance, X2 de Pearson y Métricas pseudo R2

Se plantea el uso de una clase `MetriricsReporting` para facilitar el reporting de los dos modelos.
- Cuando se instancia la clase hay que pasar como argumento el modelo en cuestión (logístico o probit)
- Uso del método `bondad_ajuste_deviance_chi_square_pseudo_r2` para visualizar el reporting

In [12]:
class MetricsReporting:
    """
    Clase para realizar el reporte de varias métricas de bondad de ajuste (deviance, chi-cuadrado y pseudo R2)

    Attributes
    ----------
    modelo : modelo realizado previamente (regresión logística o regresión probit)

    Métodos
    -------
    bondad_ajuste_deviance_chi_square_pseudo_r2 : este método es el que proporciona el reporte
    """
    
    def __init__(self, modelo: GLMResultsWrapper):
        self.modelo = modelo

    def bondad_ajuste_deviance_chi_square_pseudo_r2(self, pseudo_r2: str = "mcf"):
        """
        Función para reporte de diferentes métricas de bondad de ajuste (deviance, Chi-square o pseudo-r2).
    
        En el caso de pseudo r2 la función permite McFadden o Cox-Snell.
    
        Parameters
        ----------
        modelo : modelo (logístico o probit)
        pseudo_r2 : usa "mcf" para refererirse McFadden o "cs" para Cox-Snell (default: mcf)
        """
    
        if pseudo_r2 not in ["mcf", "cs"]:
            raise ValueError(f"{pseudo_r2} no válido. Debe ser `mcf` (McFadden’s pseudo-R-squared) o `cs` (Cox-Snell likelihood ratio pseudo R-squared)")
        
        print("Deviance")
        print(f"\n value: {round(self.modelo.deviance, 2)}")
        print(f"\n p-value: {1 - self.__pchisq(self.modelo.deviance, self.modelo.df_resid)}")
    
        print("")
        print("")
        
        print("Chi-cuadrado")
        print(f"\n value: {round(self.modelo.pearson_chi2, 2)}")
        print(f"\n p-value: {1 - self.__pchisq(self.modelo.pearson_chi2, self.modelo.df_resid)}")
    
        print("")
        print("")
        
        print("Pseudo R2 (adj)")
        print(f"\n {pseudo_r2}: {round(self.modelo.pseudo_rsquared(kind=pseudo_r2), 4)}")

    def __pchisq(self, q, df, ncp=0):
        """
        Calcula la distribución chi-cuadrado acumulada
        """
        from scipy.stats import chi2, ncx2
        if ncp == 0:
            result = chi2.cdf(x=q,df=df,loc=0,scale=1)
        else:
            result = ncx2.cdf(x=q, df=df, nc=ncp, loc=0, scale=1)
        return result

- Modelo Logístico

In [13]:
reporting = MetricsReporting(modelo=logit) # clase instanciada 
reporting.bondad_ajuste_deviance_chi_square_pseudo_r2() # pseudo r2 de mcfadden por defecto

Deviance

 value: 14771.41

 p-value: 1.0


Chi-cuadrado

 value: 24428.21

 p-value: 0.0


Pseudo R2 (adj)

 mcf: 0.2855


- Modelo Probit

In [14]:
reporting = MetricsReporting(modelo=probit) # clase instanciada 
reporting.bondad_ajuste_deviance_chi_square_pseudo_r2() # pseudo r2 de mcfadden por defecto

Deviance

 value: 14795.76

 p-value: 1.0


Chi-cuadrado

 value: 684570.02

 p-value: 0.0


Pseudo R2 (adj)

 mcf: 0.2843
